In [2]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [83]:
out_.columns

Index(['t', 'raw_acceleration_x', 'raw_acceleration_y', 'raw_acceleration_z',
       'rotation_speed_x', 'rotation_speed_y', 'rotation_speed_z',
       'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z', 't',
       'raw_acceleration_x', 'raw_acceleration_y', 'raw_acceleration_z',
       'rotation_speed_x', 'rotation_speed_y', 'rotation_speed_z',
       'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z'],
      dtype='object')

In [80]:
columns_names = []
for feature in int_.columns:
    for stat_ in stats:
        for i in quartiles:
            columns_names.append("Q",i,"_",feature+"_"+stat_)
to_test.columns = columns_names+list(to_test.columns[-2:])

Index(['t', 'raw_acceleration_x', 'raw_acceleration_y', 'raw_acceleration_z',
       'rotation_speed_x', 'rotation_speed_y', 'rotation_speed_z',
       'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z', 't',
       'raw_acceleration_x', 'raw_acceleration_y', 'raw_acceleration_z',
       'rotation_speed_x', 'rotation_speed_y', 'rotation_speed_z',
       'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z'],
      dtype='object')

In [4]:
import os
directory = 'data/v_config1-lcb'#group3/config_1'

def load_time_segments(directory, n_segments=4):
    all_data=[]
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        data = pd.read_csv(f)

        # on ajoute les stats des differents segments de l'aquisition
        n = len(data)//n_segments   # nbe de points par quartiers
        quartiles_ =[]
        for i in range(n_segments):
            quart_ = data[i*n:(i+1)*n]#.drop(columns='t')
            int_ = quart_.describe().T.drop(columns='count').T
            out_ = pd.DataFrame(pd.concat([int_.iloc[0], int_.iloc[1]], axis=0)).T
            quartiles_.append(out_)


        to_concat=[]
        for i in range(n_segments):
            to_concat.append(quartiles_[i])


        # on ajoute les stats de la serie temporelle complete
        u = data.describe().T.drop(columns='count').T
        for i in range(u.shape[0]):
            out2_ = pd.DataFrame(pd.concat([u.iloc[0], u.iloc[1]], axis=0)).T
            to_concat.append(out2_)


        number = pd.DataFrame(pd.concat(to_concat, axis=1))
        number['label'] = filename[0]
        all_data.append(number)

    return pd.concat(all_data, axis=0)
to_test = load_time_segments(directory, n_segments=4)
to_test


,t,raw_acceleration_x,raw_acceleration_y,raw_acceleration_z,rotation_speed_x,rotation_speed_y,rotation_speed_z,magnetic_field_x,magnetic_field_y,magnetic_field_z,...,raw_acceleration_x,raw_acceleration_y,raw_acceleration_z,rotation_speed_x,rotation_speed_y,rotation_speed_z,magnetic_field_x,magnetic_field_y,magnetic_field_z,label
0,23.931120,9.810526,-0.181053,-0.762105,-0.011053,0.026316,0.005789,-47.001579,19.297368,0.153684,...,4.821092,2.013747,1.970886,0.427159,1.061095,0.427764,2.292275,4.294027,10.024040,v
0,133.143997,9.960000,0.023000,-1.454500,0.026000,0.062000,0.130500,-48.079500,17.760000,-2.259500,...,4.994937,4.946531,3.469076,0.831674,1.386727,0.826707,1.720843,9.499404,8.581532,v
0,264.436068,9.131429,3.486071,0.064643,0.007857,0.012143,-0.001786,-48.972857,1.881071,-7.575357,...,2.953571,2.951886,3.740314,1.037891,0.671157,0.765532,2.097428,6.053845,7.778235,v
0,71.370835,8.288000,-4.862800,0.623600,0.260400,-0.119600,-0.064000,-27.427600,34.507600,-16.610800,...,6.765224,5.332415,4.031470,1.336040,1.397092,0.724857,6.104529,4.132449,10.309305,v
0,19.329814,9.855385,-0.301154,0.495769,-0.038846,0.073462,-0.041538,-44.323077,23.907692,-9.066923,...,2.089984,3.902031,1.514411,0.805799,0.327432,0.266053,1.851233,5.907518,6.659081,v
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,132.161252,9.674286,-0.066190,-0.059524,0.006667,0.018095,-0.055238,-47.787143,19.576667,-7.881429,...,4.924538,6.350463,4.818311,1.316642,0.966595,0.909793,5.741658,4.648246,9.627117,v
0,30.786441,10.369200,0.348800,0.335600,-0.150800,0.262000,-0.025200,-45.407200,22.985600,-7.894000,...,2.676173,3.612171,1.841000,0.943571,0.462318,0.335644,1.632816,5.881943,6.976105,v
0,227.064151,8.217500,5.575000,0.245625,-0.006250,0.020625,0.037500,-50.982500,-7.456250,-5.094375,...,6.844509,6.935323,6.270272,1.143436,1.641612,1.390144,2.104048,4.435704,9.423506,v
0,124.018163,9.908500,0.521000,-0.882500,0.011000,0.003500,-0.009500,-47.342500,17.111000,-0.080000,...,3.272555,3.435817,3.598820,1.024412,1.043442,0.612315,3.339033,3.782817,9.963763,v


In [68]:
def linearize(u):
    all=[]
    for line in range(len(u)):
        all.append(u.iloc[line])
    return pd.concat(all, axis=0).T
import os

def double_integrate(df, col):
    df.sort_values(by='t', inplace=True)
    velocities = df[col].cumsum()
    positions = velocities.cumsum()
    return positions



def load_data(directory='data/config1', drop_='t'):
    all_data=[]
    for filename in os.listdir(directory):
        # lecture fichier
        f = os.path.join(directory, filename)
        data = pd.read_csv(f)
        # ajout des positions
        data['pos_x'] = double_integrate(data,'raw_acceleration_x')
        data['pos_y'] = double_integrate(data,'raw_acceleration_y')
        data['pos_z'] = double_integrate(data,'raw_acceleration_z')
        # ajout des puissances
        data['rotation_cubic_x'] = data['rotation_speed_x']**3
        data['rotation_cubic_y'] = data['rotation_speed_y']**3
        data['rotation_cubic_z'] = data['rotation_speed_z']**3
        # feature selection / engineering 
        _ = data.drop(columns=drop_)
        u = _.describe().T.drop(['count'], axis=1)  # on enlève la colonne temps 
        number = pd.DataFrame(linearize(u)).T
        # get correct label
        if directory == 'data/config1':
            number['label'] = filename[0]
        elif directory == 'data/h_config1-lcb':
            number['label'] = filename[2]
            number['iteration'] = int(filename.split('_')[2].split('.')[0])
        elif directory == 'data/v_config1-lcb':
            number['label'] = int(filename[10])
            number['iteration'] = int(filename.split('_')[3].split('.')[0])
        elif directory == 'data/goupe1 - groupe2':
            number['iteration'] = int(filename.split('_')[2].split('.')[0])
        else :
            number['label'] = filename[0] 
        # concatenate
        all_data.append(number)   
    return pd.concat(all_data), _.columns, u.columns


In [69]:

directories = ['data/v_config1-lcb', # sur un plan 2D sur un mur, 4 personnes / 5 chiffre / pattern identique
               'data/h_config1-lcb', # sur un plan 2D sur ue table, 4 personnes / 5 chiffre / pattern identique
               'data/groupe1 - groupe2',   # dans tous les plans, 4 + 4 personnes / 5 chiffres / differents pattenrs 
               'data/group3/config_1',    # sur un plan 2D sur une table, 4 personnes / 5 chiffres / pattern identique
               'data/group3/config_3',    # pareil avec config Quaternions,
               'data/config1']             # 2D table / 1 personne fait 20 fois chaque chiffre ==> biais apprentissage

to_test, features, stats = load_data(directories[0])


In [70]:
columns_names = []
for feature in features:
    for stat_ in stats:
        columns_names.append(feature+"_"+stat_)
to_test.columns = columns_names+list(to_test.columns[-2:])
    
#mportance

# new_column_names = [('mean_' + to_test.columns[i]) for i in range(9)] + [('std_' + to_test.columns[i]) for i in range(9)] + ['label']
# to_test.columns= new_column_names

In [64]:
# merge
data1 = pd.read_csv('data/df_nb_sign_changes_v.csv')  
variables = ['label','iteration','nb_sign_changes_acceleration_x', 'nb_sign_changes_acceleration_y',
       'nb_sign_changes_acceleration_z']
to_test2 = to_test.merge(data1[variables], on=['label','iteration'], how='inner').drop(columns='iteration', inplace=True)

data1 = pd.read_csv('data/df_nb_sign_changes_h.csv')  
variables = ['label','iteration','nb_sign_changes_acceleration_x', 'nb_sign_changes_acceleration_y',
       'nb_sign_changes_acceleration_z']
to_test2 = to_test.merge(data1[variables], on=['label','iteration'], how='inner').drop(columns='iteration', inplace=True)


# enleve la colonne iteratiuon
if 'iteration' in to_test:
    to_test.drop(columns='iteration', inplace=True)